# Day 2 — Data Cleaning & Preprocessing

## Objective
Based on the issues identified during Day 1 data profiling, this phase focuses on improving dataset quality by cleaning inconsistencies, handling placeholder values, treating outliers, and preparing variables for analysis.

## Issues Identified in Day 1
- Presence of "unknown" values in categorical columns
- Potential outliers in numeric columns
- Columns with very high unknown proportions
- Need for feature preparation before analysis

This notebook performs systematic preprocessing steps to resolve these issues.

In [1]:
import pandas as pd
df = pd.read_csv("day1_output.csv")

In [2]:
#Fix Data Types
df["age"] = df["age"].astype(int)
df["balance"] = df["balance"].astype(int)

df.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

All numerical and categorical columns were checked and corrected to ensure accurate analysis. Proper datatypes prevent calculation errors and improve performance.

In [3]:
#Handling Missing values

for col in df.select_dtypes(include="object"):
    print(col, df[col].unique())

    df["job"].replace("unknown", pd.NA, inplace=True)
df["education"].replace("unknown", pd.NA, inplace=True)
df["contact"].replace("unknown", pd.NA, inplace=True)
df["poutcome"].replace("unknown", pd.NA, inplace=True)

job ['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'retired' 'admin.' 'services' 'self-employed' 'unemployed' 'housemaid'
 'student']
marital ['married' 'single' 'divorced']
education ['tertiary' 'secondary' 'unknown' 'primary']
default ['no' 'yes']
housing ['yes' 'no']
loan ['no' 'yes']
contact ['unknown' 'cellular' 'telephone']
month ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'jan' 'feb' 'mar' 'apr' 'sep']
poutcome ['unknown' 'failure' 'other' 'success']
y ['no' 'yes']


Categorical variables were standardized by converting text to lowercase and removing extra spaces to ensure consistency across categories.

In [4]:
#Remove Duplicates
df.duplicated().sum()
df = df.drop_duplicates()

Duplicate records were checked and removed to prevent bias in analysis.

In [5]:
#Handle Missing Values
df = df.replace("unknown", pd.NA)

df["job"].fillna(df["job"].mode()[0], inplace=True)
df["education"].fillna(df["education"].mode()[0], inplace=True)

df["contact"].fillna("not_specified", inplace=True)
df.drop("poutcome", axis=1, inplace=True)

df.isna().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
y            0
dtype: int64

Placeholder values such as "unknown" were treated as missing values and replaced using mode imputation for categorical columns. This preserves dataset size while maintaining realistic distributions.

In [6]:
#Outlier Detection (IQR Method)

def detect_outliers(col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    return df[(df[col] < lower) | (df[col] > upper)]

detect_outliers("balance")

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,y
34,51,management,married,tertiary,no,10635,yes,no,not_specified,5,may,336,1,-1,0,no
65,51,management,married,tertiary,no,6530,yes,no,not_specified,5,may,91,1,-1,0,no
69,35,blue-collar,single,secondary,no,12223,yes,yes,not_specified,5,may,177,1,-1,0,no
70,57,blue-collar,married,secondary,no,5935,yes,yes,not_specified,5,may,258,1,-1,0,no
186,40,services,divorced,secondary,no,4384,yes,no,not_specified,5,may,315,1,-1,0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45164,35,services,married,tertiary,no,4655,no,no,cellular,9,nov,111,2,-1,0,no
45181,46,blue-collar,married,secondary,no,6879,no,no,cellular,15,nov,74,2,118,3,no
45185,60,services,married,tertiary,no,4256,yes,no,cellular,16,nov,200,1,92,4,yes
45191,75,retired,divorced,tertiary,no,3810,yes,no,cellular,16,nov,262,1,183,1,yes


Outliers were detected using the IQR method. Extreme values outside the acceptable range were removed to prevent distortion in statistical analysis and modeling.

In [11]:
#Transformations

import numpy as np
df["balance"] = np.log1p(df["balance"])

df["balance"].skew()

np.float64(-2.534964007694962)

Log transformation was applied to skewed numeric variables to normalize their distribution and improve statistical reliability.

In [8]:
import os
os.makedirs("data/interim", exist_ok=True)

In [9]:
df.to_csv("data/interim/cleaned_day2.csv", index=False)

## Day 2 Summary

Data cleaning and preprocessing were successfully completed. Key improvements include:
- standardized categorical values
- handled placeholder missing values
- removed duplicates
- treated outliers
- transformed skewed variables

The dataset is now clean, consistent, and ready for exploratory analysis.